In [ ]:
import requests
import pandas as pd
import time

# URL de l'API IMDb
url = "https://graph.imdbapi.dev/v1"

nodes_df = pd.read_csv('data/nodes_1980_to_2000.csv')

In [3]:
def get_actor_nationality(actor_id):
    query = f"""
    query {{
        name(id: "{actor_id}") {{
            id
            display_name
            birth_location
        }}
    }}
    """
    response = requests.post(url, json={'query': query})
    
    if response.status_code == 200:
        data = response.json()
        try:
            return data['data']['name']['birth_location']
        except KeyError:
            return "Inconnu"
    else:
        print(f"Erreur {response.status_code} pour l'ID {actor_id}")
        return "Inconnu"

In [7]:
def get_country_from_location(location):
    if location:
        # Extraire le pays (dernier élément après la dernière virgule)
        return location.split(",")[-1].strip()
    return "Inconnu"

In [ ]:

# Ajouter une nouvelle colonne pour la nationalité
nationalities = []

# Récupérer la nationalité pour chaque acteur
for idx, row in nodes_df.iterrows():
    if row['category'] == 'actor':
        try: 
            actor_id = row['Id']
            nationality = get_country_from_location(get_actor_nationality(actor_id))
            nationalities.append(nationality)
            
            # Pause pour éviter de surcharger l'API
            time.sleep(0.01)
        except Exception as e:
            print(f"Erreur pour l'acteur {actor_id}: {e}")
            nationalities.append("Inconnu")
    else:
        # Pour les non-acteurs, ajouter une valeur vide ou par défaut
        nationalities.append("Inconnu")

# Ajouter les nationalités au DataFrame
nodes_df['Nationality'] = nationalities

# Sauvegarder le fichier enrichi
nodes_df.to_csv('enriched/enriched_nodes_1980_to_2000.csv', index=False)

print("Données enrichies avec les nationalités et fichier sauvegardé.")
